In [ ]:
import numpy as np
import cv2
from model import Net, CNNModel, MLP
from model import *
import torch.optim as optim
import torch
import torchvision

import numpy as np
import torch
from sklearn.model_selection import train_test_split
from os import walk
from tqdm import tqdm

from data_parser import get_torch_tensors
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

In [ ]:
# # Generate torch data
# root = '/l/users/20020038/poro_files/'
# train_x, train_y_poro, train_y_perm, test_x, test_y_poro, test_y_perm = get_torch_tensors(root, xyz_splits=[100,100,100], size=10)

# print(train_x.shape)

In [ ]:
# # Select torch data
# k = np.random.randint(0, 2000000, 100000)
# train_x_sample = train_x[k]
# train_y_poro_sample = train_y_poro[k]
# train_y_perm_sample = train_y_perm[k]
# test_x_sample = test_x[k]
# test_y_poro_sample = test_y_poro[k]
# test_y_perm_sample = test_y_perm[k]

In [ ]:
root = '/l/users/20020038/poro_files/'
# # Save data
# torch.save(train_x_sample, f'{root}saved_tensors/train_x_sample.pt')
# torch.save(train_y_poro_sample, f'{root}saved_tensors/train_y_poro_sample.pt')
# torch.save(train_y_perm_sample, f'{root}saved_tensors/train_y_perm_sample.pt')
# torch.save(test_x_sample, f'{root}saved_tensors/test_x_sample.pt')
# torch.save(test_y_poro_sample, f'{root}saved_tensors/test_y_poro_sample.pt')
# torch.save(test_y_perm_sample, f'{root}saved_tensors/test_y_perm_sample.pt')

# Load data
train_x_sample = torch.load(f'{root}saved_tensors/train_x_sample.pt')
train_y_poro_sample = torch.load(f'{root}saved_tensors/train_y_poro_sample.pt')
train_y_perm_sample = torch.load(f'{root}saved_tensors/train_y_perm_sample.pt')
test_x_sample = torch.load(f'{root}saved_tensors/test_x_sample.pt')
test_y_poro_sample = torch.load(f'{root}saved_tensors/test_y_poro_sample.pt')
test_y_perm_sample = torch.load(f'{root}saved_tensors/test_y_perm_sample.pt')

In [ ]:
# Selected indices
k = 100000
# train_inp, train_out = train_in[:k], train_o[:k]
# test_inp, test_out = test_in[:k], test_o[:k]

In [ ]:
# batch_size=128

# train = torch.utils.data.TensorDataset(train_inp, train_out)
# test = torch.utils.data.TensorDataset(test_inp, test_out)

# train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=False)
# test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)

# print('number iteration', int(k/batch_size))

In [ ]:
# Create dataloader from tensors
batch_size=128

train_out = torch.zeros((k, 1))
train_out[:,0] = train_y_poro_sample
# train_out[:,0] = train_y_perm_sample

test_out = torch.zeros((k, 1))
test_out[:,0] = test_y_poro_sample
# test_out[:,0] = test_y_perm_sample

train_inp = train_x_sample
test_inp = test_x_sample

train = torch.utils.data.TensorDataset(train_inp, train_out)
test = torch.utils.data.TensorDataset(test_inp, test_out)

train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)

print('number iteration', int(k/batch_size))

In [ ]:
train_inp.shape

In [ ]:
shape = train_inp.shape
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# net = MLP(1000).to(device) # select mlp model
# net = CNNModel().to(device) # select cnn model
net = Net(shape[1:], batch_size).to(device) # select transformer model
train_losses = []
test_losses = []
train_r2 = []
test_r2 = []

# Training setup
lr = 0.00001
epochs = 100
criterion = torch.nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)

# First train loss 
running_loss = 0.0
for i, data in tqdm(enumerate(train_loader, 0)):

    inputs, labels = data
    # inputs = inputs.unsqueeze(1)
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    optimizer.step()

    running_loss += loss.item()

train_losses.append(running_loss/len(train_loader))
o, l = outputs.cpu().detach().numpy().reshape(-1,1), labels.cpu().detach().numpy().reshape(-1,1)
r2 = r2_score(l, o)
train_r2.append(r2)
print('train error', running_loss/i)

# First validation loss
running_loss = 0.0
for i, data in tqdm(enumerate(test_loader, 0)):

    inputs, labels = data
    # inputs = inputs.unsqueeze(1)
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = net(inputs)
    loss = criterion(outputs, labels)

    running_loss += loss.item()

test_losses.append(running_loss/len(test_loader))
o, l = outputs.cpu().detach().numpy().reshape(-1,1), labels.cpu().detach().numpy().reshape(-1,1)
r2 = r2_score(l, o)
test_r2.append(r2)
print('test error', running_loss/i)

# Train model
for epoch in range(epochs):
 
    running_loss = 0.0
    
    for i, data in tqdm(enumerate(train_loader, 0)):
        
        inputs, labels = data
        # inputs = inputs.unsqueeze(1)
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    train_losses.append(running_loss/len(train_loader))
    o, l = outputs.cpu().detach().numpy().reshape(-1,1), labels.cpu().detach().numpy().reshape(-1,1)
    r2 = r2_score(l, o)
    train_r2.append(r2)
    
    if epoch % 10 == 0:
        print('epoch', epoch)
        print('train error', running_loss/i)
    
    running_loss = 0.0
    
    for i, data in tqdm(enumerate(test_loader, 0)):
        
        inputs, labels = data
        # inputs = inputs.unsqueeze(1)
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        loss = criterion(outputs, labels)

        running_loss += loss.item()
        
    test_losses.append(running_loss/len(test_loader))
    o, l = outputs.cpu().detach().numpy().reshape(-1,1), labels.cpu().detach().numpy().reshape(-1,1)
    r2 = r2_score(l, o)
    test_r2.append(r2)
    
    if epoch % 10 == 0:
        print('test error', running_loss/i)

In [ ]:
# all_outputs = []
# all_labels = []

# for i, data in tqdm(enumerate(train_loader, 0)):

#     inputs, labels = data
#     # inputs = inputs.unsqueeze(1)
#     inputs, labels = inputs.to(device), labels.to(device)
#     outputs = net(inputs)
#     loss = criterion(outputs, labels)
#     optimizer.step()

#     o, l = outputs.cpu().detach().numpy().reshape(-1,1), labels.cpu().detach().numpy().reshape(-1,1)
#     all_outputs.append(o)
#     all_labels.append(l)
#     # print('outputs', outputs.flatten())
#     # print('labels', labels.flatten())
#     # break

In [ ]:
# r2_score(np.concatenate(all_labels).flatten(), np.concatenate(all_outputs).flatten())

In [ ]:
# test_y_poro_sample mean = 0.7; max = 1; min = 0; std = 0.4
# test_y_perm_sample mean = 0.7; max = 1; min = 0; std = 0.4

In [ ]:
train_losses = np.array(train_losses)
test_losses = np.array(test_losses)
train_r2 = np.array(train_r2)
test_r2 = np.array(test_r2)

np.save('logs/' + 'train_losses', train_losses)
np.save('logs/' + 'test_losses', test_losses)
np.save('logs/' + 'train_r2', train_r2)
np.save('logs/' + 'test_r2', test_r2)

In [ ]:
# Save data
def draw(array, label, title):
    epochs = range(1,len(array)+1)
    plt.plot(epochs, array, 'g', label=label)
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('logs/' + label + '.pdf')
    plt.close()
    
    
draw(train_losses, 'Training Loss', 'Training Loss')
draw(test_losses, 'Test Loss', 'Test Loss')
draw(train_r2, 'Training R2', 'Training R2')
draw(test_r2, 'Test R2', 'Test R2')